In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers.legacy import SGD
import random

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Load and process the data
words = []  # List to hold all words
classes = []  # List to hold all classes (intents)
documents = []  # List to hold pairs of words and corresponding intent
ignore_words = ['?', '!']  # Words to ignore
data_file = open('data.json').read()  # Read the data from the JSON file
intents = json.loads(data_file)  # Load the JSON data
#Tokenize and Lemmatize Words
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the pattern
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add the tokenized pattern and corresponding intent to documents
        documents.append((w, intent['tag']))

        # Add the intent to classes if it's not already there
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize (reduce words to their base form) and convert to lowercase, then remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# Sort the classes
classes = sorted(list(set(classes)))
# Print the number of documents, classes, and unique words
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Save words and classes to files
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

# Create training data
training = []
# Create an empty array for the output with a zero for each class
output_empty = [0] * len(classes)
# Create the training set, bag of words for each sentence
for doc in documents:
    # Initialize the bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Create the bag of words array with 1 if the word is found in the current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Create the output row with a 0 for each class and a 1 for the current class
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Shuffle the training data and convert it to a numpy array
random.shuffle(training)
training = np.array(training, dtype=object)  # dtype=object to handle sequences of different lengths

# Create train and test lists. X - patterns, Y - intents
train_x = np.array([element[0] for element in training])
train_y = np.array([element[1] for element in training])

print("Training data created")

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))  # First layer with 128 neurons
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(64, activation='relu'))  # Second layer with 64 neurons
model.add(Dropout(0.5))  # Dropout layer to prevent overfitting
model.add(Dense(len(train_y[0]), activation='softmax'))  # Output layer with softmax activation

# Compile the model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit the model (train it)
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save the trained model
model.save('model.h5')

print("Model created")


193 documents
25 classes ['appointment_confirmation', 'appointment_reschedule', 'billing_inquiry', 'book_appointment', 'cancel_appointment', 'doctor_availability', 'emergency_contact', 'emergency_services', 'get_directions', 'goodbye', 'greeting', 'health_checkup_packages', 'hospital_hours', 'insurance_inquiry', 'location_query', 'medical_records', 'prescription_refill', 'provide_date', 'provide_doctor_name', 'provide_specialist_date', 'provide_specialist_type', 'specialist_consultation', 'statement_ok', 'test_results', 'thanks']
185 unique lemmatized words ["'s", ',', '15th', '24/7', '3', 'a', 'about', 'accept', 'accepted', 'access', 'address', 'afternoon', 'ajay', 'alright', 'alrighty', 'an', 'anyone', 'appointment', 'appreciate', 'appreciated', 'are', 'at', 'availability', 'available', 'awesome', 'be', 'bill', 'billing', 'book', 'booking', 'by', 'bye', 'can', 'cancel', 'cardiologist', 'care', 'catch', 'change', 'chatting', 'check', 'checkup', 'close', 'confirm', 'confirmation', 'con

39/39 [==============================] - 0s 1ms/step - loss: 0.2300 - accuracy: 0.9016
Epoch 60/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2850 - accuracy: 0.8912
Epoch 61/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2652 - accuracy: 0.8964
Epoch 62/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2993 - accuracy: 0.8860
Epoch 63/200
39/39 [==============================] - 0s 1ms/step - loss: 0.3704 - accuracy: 0.8290
Epoch 64/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2236 - accuracy: 0.9171
Epoch 65/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2855 - accuracy: 0.8601
Epoch 66/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2940 - accuracy: 0.8601
Epoch 67/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2259 - accuracy: 0.9067
Epoch 68/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2708 - accuracy: 0.8756
Epoch 69/200


39/39 [==============================] - 0s 1ms/step - loss: 0.2171 - accuracy: 0.9067
Epoch 141/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1444 - accuracy: 0.9275
Epoch 142/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1447 - accuracy: 0.9275
Epoch 143/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1600 - accuracy: 0.9119
Epoch 144/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1784 - accuracy: 0.9275
Epoch 145/200
39/39 [==============================] - 0s 1ms/step - loss: 0.2024 - accuracy: 0.9171
Epoch 146/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1836 - accuracy: 0.9275
Epoch 147/200
39/39 [==============================] - 0s 2ms/step - loss: 0.1739 - accuracy: 0.9119
Epoch 148/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1916 - accuracy: 0.8860
Epoch 149/200
39/39 [==============================] - 0s 1ms/step - loss: 0.1571 - accuracy: 0.9326
Epoc

C:\Users\venkat naidu\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
